In [2]:
import pickle
import pandas as pd
from datetime import datetime

Load and Examine the Data

In [3]:
with open('invoices_new.pkl', 'rb') as f:
    new_data = pickle.load(f)

new_data

[{'id': '365371O',
  'created_on': '2019-05-27',
  'items': [{'item': {'id': 142446,
     'name': 'ii_142446',
     'unit_price': 182,
     'type': 1},
    'quantity': 6},
   {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1},
    'quantity': 1}]},
 {'id': 389528,
  'created_on': '2019-03-19',
  'items': [{'item': {'id': 172535,
     'name': 'ii_172535',
     'unit_price': 105,
     'type': 0},
    'quantity': 4},
   {'item': {'id': 175208, 'name': 'ii_175208', 'unit_price': 109, 'type': 0},
    'quantity': 7},
   {'item': {'id': 139970, 'name': 'ii_139970', 'unit_price': 193, 'type': 2},
    'quantity': 4},
   {'item': {'id': 156273, 'name': 'ii_156273', 'unit_price': 129, 'type': 2},
    'quantity': 6},
   {'item': {'id': 178512, 'name': 'ii_178512', 'unit_price': 108, 'type': 1},
    'quantity': 7},
   {'item': {'id': 191045, 'name': 'ii_191045', 'unit_price': 195, 'type': 1},
    'quantity': 8},
   {'item': {'id': 189841, 'name': 'ii_189841', 'unit_price': 1

In [4]:
with open('expired_invoices.txt', 'r') as f:
    expired_data = f.read()

expired_data

'305869, 377307, 391273, 385290, 331902, 397723, 343695, 340601, 347510, 325156, 379387, 348894, 322229, 326649, 377960, 325063, 369378, 319405, 383681, 367288, 356552, 394428, 381476, 381457, 349879, 343254, 366751, 330931, 363263, 326452, 379687, 356532, 383235, 313012, 368913, 352442, 315960, 351096, 331193, 392657, 352391, 389528, 338547, 379961, 337140, 323231, 371205, 378746, 307175, 340299'

DataExtractor class 

In [5]:
class DataExtractor:
    def __init__(self, new_invoices_file, expired_invoices_file):
        self.new_invoices_file = new_invoices_file
        self.expired_invoices_file = expired_invoices_file

    def load_new_invoices(self):
        with open(self.new_invoices_file, 'rb') as f:
            data = pickle.load(f)
        return data

    def load_expired_invoices(self):
        with open(self.expired_invoices_file, 'r') as f:
            expired_invoices = [int(line.strip()) for line in f.read().split(',')]
        return expired_invoices
    
    def transform_data(self):
        new_invoices = self.load_new_invoices()
        expired_invoices = self.load_expired_invoices()

        transformed_data = []
        transformed_item = {}
        invoice_total = 0
        for invoice in new_invoices:
            invoice_id = invoice['id']
            transformed_item['invoice_id'] = invoice_id
            is_expired = invoice_id in expired_invoices
            transformed_item['is_expired'] = is_expired


            try:
                created_on = datetime.strptime(invoice.get('created_on'), '%Y-%m-%d')
            except (ValueError, TypeError):
                print(f"Invalid date format in invoice: {invoice}")
                created_on = None
            
            transformed_item['created_on'] = created_on


            if 'items' in invoice:
                for item in invoice['items']:
                    unit_item = item['item']
                    value = unit_item['id']
                    transformed_item['invoiceitem_id'] = value
                    
                    value = unit_item['name']
                    transformed_item['invoiceitem_name'] = value

                    value = unit_item['type']
                    transformed_item['type'] = 'Material' if value == 0 else 'Equipment' if value == 1 else 'Service' if value == 2 else 'Other'

                    value = unit_item['unit_price']
                    transformed_item['unit_price'] = value

                    unit_price =  value
                    value = item['quantity']
                    if isinstance(value, int) == True :
                        transformed_item['total_price'] = value * unit_price

                        invoice_total += value * unit_price
                    transformed_data.append(transformed_item)

        for data in transformed_data:
            data['percentage_in_invoice'] = data['total_price'] / invoice_total


        df = pd.DataFrame(transformed_data)
        df = df[['invoice_id', 'created_on', 'invoiceitem_id', 'invoiceitem_name', 'type', 'unit_price', 'total_price', 'percentage_in_invoice', 'is_expired']]
        df['created_on'] = pd.to_datetime(df['created_on'])
        df = df.sort_values(by=['invoice_id', 'invoiceitem_id']).reset_index(drop=True)

        return df

In [6]:
# Instantiate the DataExtractor
extractor = DataExtractor('invoices_new.pkl', 'expired_invoices.txt')

# Load and print new invoices
loaded_new_invoices = extractor.load_new_invoices()
print("Loaded New Invoices:")
print(loaded_new_invoices)

# Load and print expired invoices
loaded_expired_invoices = extractor.load_expired_invoices()
print("Loaded Expired Invoices:")
print(loaded_expired_invoices)
print("Number of new invoices:", len(loaded_new_invoices))
print("Number of expired invoices:", len(loaded_expired_invoices))

Loaded New Invoices:
[{'id': '365371O', 'created_on': '2019-05-27', 'items': [{'item': {'id': 142446, 'name': 'ii_142446', 'unit_price': 182, 'type': 1}, 'quantity': 6}, {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1}, 'quantity': 1}]}, {'id': 389528, 'created_on': '2019-03-19', 'items': [{'item': {'id': 172535, 'name': 'ii_172535', 'unit_price': 105, 'type': 0}, 'quantity': 4}, {'item': {'id': 175208, 'name': 'ii_175208', 'unit_price': 109, 'type': 0}, 'quantity': 7}, {'item': {'id': 139970, 'name': 'ii_139970', 'unit_price': 193, 'type': 2}, 'quantity': 4}, {'item': {'id': 156273, 'name': 'ii_156273', 'unit_price': 129, 'type': 2}, 'quantity': 6}, {'item': {'id': 178512, 'name': 'ii_178512', 'unit_price': 108, 'type': 1}, 'quantity': 7}, {'item': {'id': 191045, 'name': 'ii_191045', 'unit_price': 195, 'type': 1}, 'quantity': 8}, {'item': {'id': 189841, 'name': 'ii_189841', 'unit_price': 171, 'type': 1}, 'quantity': 4}]}, {'id': 341762, 'created_on': '2019-06

In [7]:
data_extractor = DataExtractor('invoices_new.pkl', 'expired_invoices.txt')
df = data_extractor.transform_data()
df.head()

Invalid date format in invoice: {'id': 366751, 'created_on': '2019-02-30', 'items': [{'item': {'id': 175333, 'name': 'ii_175333', 'unit_price': 192, 'type': 3}, 'quantity': 5}, {'item': {'id': 106160, 'name': 'ii_106160', 'unit_price': 107, 'type': 2}, 'quantity': 1}, {'item': {'id': 160248, 'name': 'ii_160248', 'unit_price': 118, 'type': 3}, 'quantity': 6}, {'item': {'id': 159445, 'name': 'ii_159445', 'unit_price': 144, 'type': 1}, 'quantity': 5}]}
Invalid date format in invoice: {'id': 375592, 'created_on': '2019-02-30', 'items': [{'item': {'id': 157027, 'name': 'ii_157027', 'unit_price': 187, 'type': 1}, 'quantity': -1}, {'item': {'id': 185821, 'name': 'ii_185821', 'unit_price': 157, 'type': 2}, 'quantity': 3}]}


,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,percentage_in_invoice,is_expired
0,377960,2019-02-22,144902,ii_144902,Service,188,1316,0.004,True
1,377960,2019-02-22,144902,ii_144902,Service,188,1316,0.004,True
2,377960,2019-02-22,144902,ii_144902,Service,188,1316,0.004,True
3,377960,2019-02-22,144902,ii_144902,Service,188,1316,0.004,True
4,377960,2019-02-22,144902,ii_144902,Service,188,1316,0.004,True


In [9]:
df.to_csv("invoice-data-extractor.csv", index = False)